# Assignment Cassandra

### Importing Packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Creating Session connection to cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Drop keyspace m14 if it exists

In [3]:
session.execute("DROP KEYSPACE IF EXISTS m14")

### Describing all keyspaces

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Creating Keyspace m14

In [5]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

### Describing All Keyspaces to check m14

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Creating an inventory table in m14

In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    SKU TEXT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT, 
    PRIMARY KEY(SKU)
);
""")

### Reading CSV file

In [8]:
df = pd.read_csv('inventoryinfo.csv')
df

,SKU,name,description,warehouse_num
0,TSLA001,Model S,Model S is Tesla flagship electric sedan.,101
1,TSLA002,Model 3,Model 3 is Tesla most affordable electric car.,201
2,TSLA003,Model X,Model X is an electric SUV known for its falco...,321
3,TSLA004,Model Y,Model Y is a compact electric SUV with spaciou...,123
4,TSLA005,Model Plaid S,Model Plaid S is the high-performance version ...,435
5,TSLA006,Model Plaid X,Model Plaid X is the high-performance version ...,150
6,TSLA007,Cyber Truck,"Cybertruck is an all-electric, futuristic pick...",123


### Printing the values from DF

In [9]:
for index, row in df.iterrows():
    print(f"SKU = {row.SKU}, name ={row.name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

SKU = TSLA001, name =0, description = Model S is Tesla flagship electric sedan., warehouse_num = 101
SKU = TSLA002, name =1, description = Model 3 is Tesla most affordable electric car., warehouse_num = 201
SKU = TSLA003, name =2, description = Model X is an electric SUV known for its falcon-wing doors and versatile interior., warehouse_num = 321
SKU = TSLA004, name =3, description = Model Y is a compact electric SUV with spacious seating and advanced autopilot features., warehouse_num = 123
SKU = TSLA005, name =4, description = Model Plaid S is the high-performance version of the Model S., warehouse_num = 435
SKU = TSLA006, name =5, description = Model Plaid X is the high-performance version of the Model X., warehouse_num = 150
SKU = TSLA007, name =6, description = Cybertruck is an all-electric, futuristic pickup truck designed by Tesla, Inc., warehouse_num = 123


### Inserting the values in to m14.inventory table

In [10]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ('{row['SKU']}', '{row['name']}', '{row['description']}', {row['warehouse_num']});
    """)
    
    session.execute(f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ('{row['SKU']}', '{row['name']}', '{row['description']}', {row['warehouse_num']});
    """)



        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ('TSLA001', 'Model S', 'Model S is Tesla flagship electric sedan.', 101);
    

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ('TSLA002', 'Model 3', 'Model 3 is Tesla most affordable electric car.', 201);
    

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ('TSLA003', 'Model X', 'Model X is an electric SUV known for its falcon-wing doors and versatile interior.', 321);
    

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ('TSLA004', 'Model Y', 'Model Y is a compact electric SUV with spacious seating and advanced autopilot features.', 123);
    

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ('TSLA005', 'Model Plaid S', 'Model Plaid S is the high-performance version of the Model S.', 435);
    

        IN

### Selecting all the data from m14.invetory table

In [11]:
rows = session.execute("select (SKU, name, description, warehouse_num) from m14.inventory")
for row in rows:
    print(f"SKU={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


SKU=TSLA007, name=Cyber Truck, description=Cybertruck is an all-electric, futuristic pickup truck designed by Tesla, Inc., warehouse_num=123
SKU=TSLA003, name=Model X, description=Model X is an electric SUV known for its falcon-wing doors and versatile interior., warehouse_num=321
SKU=TSLA006, name=Model Plaid X, description=Model Plaid X is the high-performance version of the Model X., warehouse_num=150
SKU=TSLA004, name=Model Y, description=Model Y is a compact electric SUV with spacious seating and advanced autopilot features., warehouse_num=123
SKU=TSLA002, name=Model 3, description=Model 3 is Tesla most affordable electric car., warehouse_num=201
SKU=TSLA001, name=Model S, description=Model S is Tesla flagship electric sedan., warehouse_num=101
SKU=TSLA005, name=Model Plaid S, description=Model Plaid S is the high-performance version of the Model S., warehouse_num=435


### Creating a index named warehouse_num for warehouse_num column

In [12]:
session.execute("""
    CREATE INDEX IF NOT EXISTS warehouse_num ON m14.inventory (warehouse_num);
""")

### selecting name and description with warehouse_num = 123 as filter

In [13]:
rows = session.execute("select (name, description) from m14.inventory where warehouse_num = 123 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}")

Cyber Truck, Cybertruck is an all-electric, futuristic pickup truck designed by Tesla, Inc.
Model Y, Model Y is a compact electric SUV with spacious seating and advanced autopilot features.
